In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(columns=['RowNumber','CustomerId','Surname'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
with open('label_encoder_gender.pkl','rb') as file: 
    gender_encoder = pickle.load(file)
data['Gender'] =  gender_encoder.transform(data['Gender'])
data.head(10)

c:\Users\ASUS\Desktop\University of Luxembourg\Udemy\Learning GenAI\NLP\Churn Classification\annclassification\venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [4]:
with open('one_hot_encoder_geo.pkl','rb') as file :
    geo_encoder = pickle.load(file)
geo = geo_encoder.transform(data[['Geography']])

c:\Users\ASUS\Desktop\University of Luxembourg\Udemy\Learning GenAI\NLP\Churn Classification\annclassification\venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
geo_df = pd.DataFrame(geo.toarray(),columns=geo_encoder.get_feature_names_out())
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [6]:
data = pd.concat([data,geo_df],axis=1)
data = data.drop('Geography',axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
from sklearn.model_selection import train_test_split
X = data.drop('Exited',axis=1)
y= data['Exited']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
print(len(X_train),len(X_test))

8000 2000


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data

array([[ 0.04286431, -1.08565613, -0.27483865, ..., -1.0032553 ,
         1.72859362, -0.57369368],
       [-2.28411809,  0.92110197, -0.08338017, ...,  0.99675526,
        -0.57850497, -0.57369368],
       [ 0.45655007,  0.92110197, -1.04067259, ..., -1.0032553 ,
        -0.57850497,  1.74309049],
       ...,
       [-1.78769518, -1.08565613, -0.08338017, ...,  0.99675526,
        -0.57850497, -0.57369368],
       [-0.6397172 ,  0.92110197,  1.54401694, ..., -1.0032553 ,
         1.72859362, -0.57369368],
       [-1.5808523 ,  0.92110197, -0.46629714, ...,  0.99675526,
        -0.57850497, -0.57369368]])

In [9]:
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape = (X_train.shape[1],))) #Input layer
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu')) #hidden layers
    model.add(Dense(1,activation='sigmoid')) #Final layer
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [13]:
model = KerasClassifier(neurons=32,layers=1,build_fn=create_model,epochs=50,batch_size=10,verbose=1)

In [18]:
params = {
    'neurons' : [16,32,64,128],
    'layers' : [1,2],
    'epochs' : [50,100]   
}

In [19]:
grid = GridSearchCV(estimator=model, param_grid=params,n_jobs=-1,cv=3)
grid_result = grid.fit(X_train,y_train)

c:\Users\ASUS\Desktop\University of Luxembourg\Udemy\Learning GenAI\NLP\Churn Classification\annclassification\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/100


800/800 [==============================] - 3s 2ms/step - loss: 107.6408 - accuracy: 0.6862
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 49.9977 - accuracy: 0.6880
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 40.3575 - accuracy: 0.6889
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 43.4557 - accuracy: 0.6954
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 34.9769 - accuracy: 0.6936
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 35.1081 - accuracy: 0.6777
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 32.3713 - accuracy: 0.6765
Epoch 8/100
800/800 [==============================] - 1s 2ms/step - loss: 36.6034 - accuracy: 0.6880
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 29.9154 - accuracy: 0.6846
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 2

In [20]:
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best score: 0.796750 using {'epochs': 100, 'layers': 2, 'neurons': 16}


#### So now we can create the model using the above params.

In [38]:
model = Sequential([
    Dense(32, activation='relu', input_shape= (X_train.shape[1],)),
    Dense(16,activation ='relu'),
    Dense(16,activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [39]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 32)                416       
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1233 (4.82 KB)
Trainable params: 1233 (4.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
early_stopping_callback = EarlyStopping(monitor="val_loss", patience = 10, restore_best_weights=True)
fit = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=early_stopping_callback)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 267.4051 - accuracy: 0.6630 - val_loss: 18.6161 - val_accuracy: 0.6820
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 43.7676 - accuracy: 0.6585 - val_loss: 75.2071 - val_accuracy: 0.7920
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 65.1777 - accuracy: 0.6660 - val_loss: 24.0032 - val_accuracy: 0.6430
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 41.0895 - accuracy: 0.6739 - val_loss: 33.5142 - val_accuracy: 0.7945
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 40.7501 - accuracy: 0.6668 - val_loss: 20.8778 - val_accuracy: 0.5350
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 41.1263 - accuracy: 0.6704 - val_loss: 27.4314 - val_accuracy: 0.6345
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 29.6562 - accuracy: 0.6787 - val_loss: 45.1051 - val_a